In [ ]:
!git clone https://github.com/pytorch/examples.git

In [ ]:
!pip install -r /content/examples/mnist/requirements.txt

In [ ]:
!python /content/examples/mnist/main.py --epochs 1

In [ ]:
os.environ["MASTER_ADDR"] = "127.0.0.1"
os.environ["MASTER_PORT"] = args.port
os.environ["WORLD_SIZE"] = args.world_size
os.environ["RANK"] = args.rank

In [ ]:
!python --version

Python 3.7.14


In [ ]:
import torch
for i in range(torch.cuda.device_count()):
    print(f'torch.cuda.get_device_properties({i}) : {torch.cuda.get_device_properties(i)}',
    f'torch.cuda.device_count() : {torch.cuda.device_count()}',
    f'torch.cuda.current_device() : {torch.cuda.current_device()}',
    f'torch.cuda.memory_allocated({i}) : {round(torch.cuda.memory_allocated(i)/1024**3,1)} GB',
    f'torch.cuda.memory_cached({i}) : {round(torch.cuda.memory_cached(i)/1024**3,1)} GB', sep='\n')

torch.cuda.get_device_properties(0) : _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)
torch.cuda.device_count() : 1
torch.cuda.current_device() : 0
torch.cuda.memory_allocated(0) : 0.0 GB
torch.cuda.memory_cached(0) : 0.0 GB


In [ ]:
import torch
"""
arg.distributed : node가 2개 이상일때 True, multi machine
                    node가 1개 : single machine

"""
 # 내용3: multiprocess 설정
if arg.distributed:
    nnode = arg.nnod if arg.nnod >1 else 1  

    if torch.cuda.is_available():
        ngpus_per_node = torch.cuda.device_count()
        world_size = ngpus_per_node * nnode
        batch_size = int(batch_size / world_size)
        workers = int((workers + world_size - 1) / world_size)

        for i in range(ngpus_per_node):
            model.cuda(i)
            model = torch.nn.parallel.DistributedDataParallel(model,device_ids=[i]) # device_ids = gpu, 0,1,2,...
            # args.gpu : 0,1,2,3...
        """
        model.cuda()
        model = torch.nn.parallel.DistributedDataParallel(model)
        # 만약에 device_ids를 따로 설정해주지 않으면, 가능한 모든 gpu를 기준으로 
            ddp가 알아서 배치사이즈와 workers를 나눠준다
        
        """
    elif args.gpu is not None:
        torch.cuda.set_device(args.gpu)
        model=model.cuda(args.gpu)
        raise NotImplementedError("Only DistributedDataParallel is supported.")
    else:
        raise NotImplementedError("Only DistributedDataparallel is supported.")

NameError: ignored

In [ ]:
criterion.cuda(arg.gpu)

In [ ]:
if arg.distributed:
    train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
else :
    train_sampler = None

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=args.batch_size,
                                           shuffle=(train_sampler is None),
                                           num_workers=args.workers,
                                           pin_memory=True,
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(val_dataset,
                                         batch_size=args.batch_size,
                                         shuffle=False,
                                         num_workers=args.workers,
                                         pin_memory=True)

In [ ]:
cudnn.benchmark = True
stime = time.time()
scaler = torch.cuda.amp.GradScaler()
for epoch in range(args.start_epoch, args.epochs):
    # 내용 6-1: train_sampler.set_epoch
    # In distributed mode, calling the set_eopch() method at the beggining of 
    # each epoch before creating the "dataloader" iterator is necessary to make
    # suffling work properly across multiple epochs. Otherwise, the same ordering will be always used.
    if args.distributed:
        train_sampler.set_epoch(epoch)

